In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
# !pip install paddlepaddle-gpu
# !pip install paddleocr
# !pip install tqdm


In [9]:
from functools import partial

In [10]:
# Import necessary libraries
import os
import pandas as pd
import requests
from tqdm import tqdm
from paddleocr import PaddleOCR
from multiprocessing import Pool
from pathlib import Path
from PIL import Image
from io import BytesIO

# Initialize PaddleOCR (ensure GPU is enabled on Kaggle)
ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=True)


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:16<00:00, 250kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:18<00:00, 546kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:15<00:00, 138kiB/s] 

[2024/09/16 02:56:04] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_ch

## Load our csv file

In [9]:
# # Load the CSV file
# df = pd.read_csv('/kaggle/input/amazon-test/test.csv')

# # Let's inspect the first few rows
# df.head()


In [10]:
# def download_image(image_link, save_folder):
#     """Download an image from a URL to the specified folder."""
#     try:
#         filename = Path(image_link).name
#         image_save_path = os.path.join(save_folder, filename)
        
#         if not os.path.exists(image_save_path):  # Avoid downloading if already exists
#             response = requests.get(image_link, timeout=10)
#             if response.status_code == 200:
#                 img = Image.open(BytesIO(response.content))
#                 img.save(image_save_path)
#         return filename  # Return the filename for tracking
#     except Exception as e:
#         return None  # Return None if there's an error


In [11]:
# def download_images(image_links, download_folder):
#     """Download all images in parallel."""
#     if not os.path.exists(download_folder):
#         os.makedirs(download_folder)
    
#     with Pool(16) as pool:  # Adjust number based on available cores
#         result = list(tqdm(pool.imap(partial(download_image, save_folder=download_folder), image_links), total=len(image_links)))
    
#     return result


In [11]:
import multiprocessing

# Get the number of CPU cores
available_cores = multiprocessing.cpu_count()
print(f"Available CPU cores: {available_cores}")


Available CPU cores: 4


In [12]:
import psutil

# Print current CPU and memory usage
print(f"CPU usage: {psutil.cpu_percent()}%")
print(f"Memory usage: {psutil.virtual_memory().percent}%")


CPU usage: 5.2%
Memory usage: 5.8%


In [13]:
# # Download images from image_link column
# image_folder = './downloaded_images'
# image_links = df['image_link'].tolist()

# # Download all images
# downloaded_files = download_images(image_links, image_folder)


## Batch Preprocessing

In [14]:
# OCR function
def extract_text_from_image(image_path):
    """Extract text from an image using PaddleOCR."""
    try:
        result = ocr.ocr(image_path, cls=True)
        text = ' '.join([line[-1][0] for line in result[0]])
        return text
    except Exception as e:
        return ""  # Return empty string if OCR fails

In [15]:
# Process images and perform OCR
def process_images_for_ocr(image_folder, image_files):
    """Perform OCR on all images in the folder."""
    ocr_results = {}

    for image_file in tqdm(image_files, total=len(image_files)):
        image_path = os.path.join(image_folder, image_file)
        
        if os.path.exists(image_path):
            ocr_text = extract_text_from_image(image_path)
            ocr_results[image_file] = ocr_text
        else:
            ocr_results[image_file] = "Image not found"
    
    return ocr_results


In [16]:
# Batch processing function
def process_in_batches(image_folder, batch_size=1000, output_folder="./ocr_output"):
    # Get a list of all image files in the folder
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    num_batches = len(image_files) // batch_size + 1
    
    for batch_num in range(num_batches):
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, len(image_files))
        batch_files = image_files[start_idx:end_idx]
        
        # Step 1: Perform OCR on the batch of images
        ocr_results = process_images_for_ocr(image_folder, batch_files)
        
        # Step 2: Create a DataFrame to store results
        batch_df = pd.DataFrame({
            'image_file': batch_files,
            'ocr_text': [ocr_results.get(image, '') for image in batch_files]
        })
        
        # Step 3: Save the batch result to a CSV file
        batch_df.to_csv(f"{output_folder}/ocr_results_batch_{batch_num}.csv", index=False)
        
        print(f"Processed batch {batch_num + 1}/{num_batches}")



<!-- add these possible outputs as well, with your strong logic, "# ML Challenge Problem Statement

## Feature Extraction from Images

In this hackathon, the goal is to create a machine learning model that extracts entity values from images. This capability is crucial in fields like healthcare, e-commerce, and content moderation, where precise product information is vital. As digital marketplaces expand, many products lack detailed textual descriptions, making it essential to obtain key details directly from images. These images provide important information such as weight, volume, voltage, wattage, dimensions, and many more, which are critical for digital stores.

### Data Description: 

The dataset consists of the following columns: 

1. **index:** An unique identifier (ID) for the data sample
2. **image_link**: Public URL where the product image is available for download. Example link - https://m.media-amazon.com/images/I/71XfHPR36-L.jpg
    To download images use `download_images` function from `src/utils.py`. See sample code in `src/test.ipynb`.
3. **group_id**: Category code of the product
4. **entity_name:** Product entity name. For eg: “item_weight” 
5. **entity_value:** Product entity value. For eg: “34 gram” 
    Note: For test.csv, you will not see the column `entity_value` as it is the target variable.

### Output Format:

The output file should be a csv with 2 columns:

1. **index:** The unique identifier (ID) of the data sample. Note the index should match the test record index.
2. **prediction:** A string which should have the following format: “x unit” where x is a float number in standard formatting and unit is one of the allowed units (allowed units are mentioned in the Appendix). The two values should be concatenated and have a space between them. For eg: “2 gram”, “12.5 centimetre”, “2.56 ounce” are valid. Few invalid cases: “2 gms”, “60 ounce/1.7 kilogram”, “2.2e2 kilogram” etc.
    Note: Make sure to output a prediction for all indices. If no value is found in the image for any test sample, return empty string, i.e, `“”`. If you have less/more number of output samples in the output file as compared to test.csv, your output won’t be evaluated. 

### File Descriptions:

*source files*

1. **src/sanity.py**: Sanity checker to ensure that the final output file passes all formatting checks. Note: the script will not check if less/more number of predictions are present compared to the test file. See sample code in `src/test.ipynb` 
2. **src/utils.py**: Contains helper functions for downloading images from the image_link.
3. **src/constants.py:** Contains the allowed units for each entity type.
4. **sample_code.py:** We also provided a sample dummy code that can generate an output file in the given format. Usage of this file is optional. 

*Dataset files*

1. **dataset/train.csv**: Training file with labels (`entity_value`).
2. **dataset/test.csv**: Test file without output labels (`entity_value`). Generate predictions using your model/solution on this file's data and format the output file to match sample_test_out.csv (Refer the above section "Output Format")
3. **dataset/sample_test.csv**: Sample test input file.
4. **dataset/sample_test_out.csv**: Sample outputs for sample_test.csv. The output for test.csv must be formatted in the exact same way. Note: The predictions in the file might not be correct

### Constraints

1. You will be provided with a sample output file and a sanity checker file. Format your output to match the sample output file exactly and pass it through the sanity checker to ensure its validity. Note: If the file does not pass through the sanity checker, it will not be evaluated. You should recieve a message like `Parsing successfull for file: ...csv` if the output file is correctly formatted.

2. You are given the list of allowed units in constants.py and also in Appendix. Your outputs must be in these units. Predictions using any other units will be considered invalid during validation.

### Evaluation Criteria

Submissions will be evaluated based on F1 score, which are standard measures of prediction accuracy for classification and extraction problems.

Let GT = Ground truth value for a sample and OUT be output prediction from the model for a sample. Then we classify the predictions into one of the 4 classes with the following logic: 

1. *True Positives* - If OUT != `""` and GT != `""` and OUT == GT
2. *False Positives* - If OUT != `""` and GT != `""` and OUT != GT
3. *False Positives* - If OUT != `""` and GT == `""`
4. *False Negatives* - If OUT == `""` and GT != `""`
5. *True Negatives* - If OUT == `""` and GT == `""` 

Then, F1 score = 2*Precision*Recall/(Precision + Recall) where:

1. Precision = True Positives/(True Positives + False Positives)
2. Recall = True Positives/(True Positives + False Negatives)

### Submission File

Upload a test_out.csv file in the Portal with the exact same formatting as sample_test_out.csv

### Appendix
 -->
```
entity_unit_map = {
  "width": {
    "centimetre",
    "foot",
    "millimetre",
    "metre",
    "inch",
    "yard"
  },
  "depth": {
    "centimetre",
    "foot",
    "millimetre",
    "metre",
    "inch",
    "yard"
  },
  "height": {
    "centimetre",
    "foot",
    "millimetre",
    "metre",
    "inch",
    "yard"
  },
  "item_weight": {
    "milligram",
    "kilogram",
    "microgram",
    "gram",
    "ounce",
    "ton",
    "pound"
  },
  "maximum_weight_recommendation": {
    "milligram",
    "kilogram",
    "microgram",
    "gram",
    "ounce",
    "ton",
    "pound"
  },
  "voltage": {
    "millivolt",
    "kilovolt",
    "volt"
  },
  "wattage": {
    "kilowatt",
    "watt"
  },
  "item_volume": {
    "cubic foot",
    "microlitre",
    "cup",
    "fluid ounce",
    "centilitre",
    "imperial gallon",
    "pint",
    "decilitre",
    "litre",
    "millilitre",
    "quart",
    "cubic inch",
    "gallon"
  }
}
```
" in def extractedEntityValue(entity_name,unit_list):
    if(entity_name=='voltage'):
        # Use list comprehension to filter and find the values with 'v' (for volts)
        volt_list = [float(x[0]) for x in unit_list if x[1].lower() == 'v']
        
        # If volt_list is not empty, return the max value, else return a default value
        if volt_list:
            return f"{max(volt_list)} volt"
        else:
            return ""  # or an appropriate default value
    
    elif(entity_name=='wattage'):
        # Use list comprehension to filter and find the values with 'v' (for volts)
        volt_list = [float(x[0]) for x in unit_list if x[1].lower() == 'w']
        
        # If volt_list is not empty, return the max value, else return a default value
        if volt_list:
            return f"{max(volt_list)} watt"
        else:
            return ""  # or an appropriate default value
    
    elif entity_name == 'item_volume':
        # Filter values with 'ml', 'oz', or 'floz' for millilitres or ounces
        volume_list = [(float(x[0]), x[1].lower()) for x in unit_list if x[1].lower() in ['ml', 'oz', 'floz']]
        
        if volume_list:
            # Get the maximum value and its unit
            max_value, unit = max(volume_list)
            
            # Convert unit to full name
            if unit == 'ml':
                unit_name = 'millilitre'
            elif unit == 'floz':
                unit_name = 'fluid ounce'
            else:
                unit_name = 'ounce'
            
            return f"{max_value} {unit_name}"
        else:
            return ""
    
    elif entity_name in ['depth', 'width', 'height']:
        import random
        # Filter values with 'mm', 'cm', or 'm' for units of measurement
        depth_list = [(float(x[0]), x[1].lower()) for x in unit_list if x[1].lower() in ['mm', 'cm', 'm']]
        
        if depth_list:
            # Randomly select one of the measurements
            idx = random.randint(0, len(depth_list) - 1)  # Use randint to get a valid index
            max_value, unit = depth_list[idx]
            
            # Convert unit to full name
            if unit == 'mm':
                unit_name = 'millimeter'
            elif unit == 'cm':
                unit_name = 'centimeter'
            elif unit == 'm':
                unit_name = 'meter'
            
            return f"{max_value} {unit_name}"
        else:
            return ""

        

    elif entity_name == 'item_weight' or entity_name == 'maximum_weight_recommendation':
        # Weight conversion hashmap
        conversion_factors = {
            "mg": 1 / 1000,  # Convert mg to grams
            "g": 1,          # grams as base unit
            "kg": 1000,      # Convert kg to grams
            "oz": 28.34,     # Convert oz to grams
            "pound": 453.62,  # Convert pound to grams

        }

        # Handle various forms of grams
        def normalize_gram_unit(unit):
            unit = unit.lower()
            if unit in ['g', 'gm', 'gms','gpack','plantago','gsm','gn','grams']:
                return 'g'
            if unit in ['pound','lbs','']:
                return 'pound'
            if unit in ['kg','kgs']:
                return 'kg'
            
            return unit

        # List of converted weights in grams and original values
        weight_list = [(float(x[0]) * conversion_factors[normalize_gram_unit(x[1])], float(x[0]), normalize_gram_unit(x[1]))
                       for x in unit_list if normalize_gram_unit(x[1]) in conversion_factors]

        
        if weight_list:
            # Get the max weight in grams, along with the original value and unit
            _, max_value, unit = max(weight_list)
            
            # Mapping of unit abbreviations to full names
            unit_full_names = {
                "mg": "milligram",
                "g": "gram",
                "kg": "kilogram",
                "oz": "ounce",
                "pound": "pound"
            }

            unit_name = unit_full_names[unit]
            return f"{max_value} {unit_name}"
        else:
            return ""
    return ""

we have extracted using pytessract so think of possible standard way of mistake and how can we best find match 
for your information, i am telling how we are fetching units, def findUnits(text):
    import re
    pattern = r"(\d+(?:\.\d+)?)\s*([A-Za-z]+)"
    matches = re.findall(pattern, text)
    return matches

In [31]:
name = 'training_19k'

In [ ]:
# Process the images in batches and save the results
process_in_batches(image_folder="/kaggle/input/train-19k/train_19k", batch_size=500, output_folder=f"./ocr_output_{name}")

  0%|          | 0/500 [00:00<?, ?it/s]

[2024/09/16 05:16:34] ppocr DEBUG: dt_boxes num : 8, elapsed : 0.055310964584350586
[2024/09/16 05:16:34] ppocr DEBUG: cls num  : 8, elapsed : 0.01650404930114746
[2024/09/16 05:16:34] ppocr DEBUG: rec_res num  : 8, elapsed : 0.04324913024902344


  0%|          | 1/500 [00:00<01:06,  7.51it/s]

[2024/09/16 05:16:34] ppocr DEBUG: dt_boxes num : 5, elapsed : 0.05435347557067871
[2024/09/16 05:16:34] ppocr DEBUG: cls num  : 5, elapsed : 0.008543252944946289
[2024/09/16 05:16:34] ppocr DEBUG: rec_res num  : 5, elapsed : 0.018040895462036133


  0%|          | 2/500 [00:00<00:57,  8.73it/s]

...

[2024/09/16 05:31:35] ppocr DEBUG: dt_boxes num : 4, elapsed : 0.05264735221862793
[2024/09/16 05:31:35] ppocr DEBUG: cls num  : 4, elapsed : 0.00799250602722168
[2024/09/16 05:31:35] ppocr DEBUG: rec_res num  : 4, elapsed : 0.01033163070678711
[2024/09/16 05:31:35] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.0387880802154541
[2024/09/16 05:31:35] ppocr DEBUG: cls num  : 1, elapsed : 0.007537126541137695
[2024/09/16 05:31:35] ppocr DEBUG: rec_res num  : 1, elapsed : 0.009741067886352539


 22%|██▏       | 108/500 [00:11<00:41,  9.36it/s]

[2024/09/16 05:31:35] ppocr DEBUG: dt_boxes num : 11, elapsed : 0.02822256088256836
[2024/09/16 05:31:35] ppocr DEBUG: cls num  : 11, elapsed : 0.016294479370117188
[2024/09/16 05:31:35] ppocr DEBUG: rec_res num  : 11, elapsed : 0.04729938507080078
[2024/09/16 05:31:35] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.05301618576049805
[2024/09/16 05:31:35] ppocr DEBUG: cls num  : 2, elapsed : 0.007704734802246094
[2024/09/16 05:31:35] ppocr DEBUG: rec_res num  : 2, elapsed : 0.009461402893066406


 22%|██▏       | 110/500 [00:11<00:41,  9.47it/s]

[2024/09/16 05:31:35] ppocr DEBUG: dt_boxes num : 5, elapsed : 0.04595065116882324
[2024/09/16 05:31:35] ppocr DEBUG: cls num  : 5, elapsed : 0.008899688720703125


In [30]:
import os
import pandas as pd
import zipfile

# Combine all CSV files in /kaggle/working/ocr_output
def combine_csv_files(output_folder=f"./ocr_output_{name}", combined_csv=f"{name}_combined_ocr_results.csv"):
    # List all CSV files in the output folder
    csv_files = [f for f in os.listdir(output_folder) if f.endswith('.csv')]
    
    # Initialize an empty list to store DataFrames
    df_list = []
    
    # Loop through each CSV file and append it to the list
    for csv_file in csv_files:
        file_path = os.path.join(output_folder, csv_file)
        df = pd.read_csv(file_path)
        df_list.append(df)
    
    # Concatenate all DataFrames into one
    combined_df = pd.concat(df_list, ignore_index=True)
    
    # Save the combined DataFrame as a single CSV file
    combined_df.to_csv(combined_csv, index=False)
    
    print(f"Combined CSV saved to {combined_csv}")

# Zip the combined CSV file
def zip_csv_file(csv_file, zip_file=f"ocr_results_{name}.zip"):
    # Create a zip file
    with zipfile.ZipFile(zip_file, 'w') as zipf:
        zipf.write(csv_file, os.path.basename(csv_file))
    
    print(f"Zipped file saved to {zip_file}")

# Combine all CSV files and zip the result
combined_csv_file = f"{name}_combined_ocr_results.csv"
combine_csv_files(output_folder=f"/kaggle/working/ocr_output_{name}", combined_csv=combined_csv_file)
zip_csv_file(combined_csv_file, zip_file=f"{name}_ocr_results.zip")


Combined CSV saved to training_2580_combined_ocr_results.csv
Zipped file saved to training_2580_ocr_results.zip


In [22]:
print("Hello world")

Hello world
